In [1]:
import json
from prettytable import PrettyTable
import json
import os
import random
import pandas as pd

# Read the jsonl file and convert it to a JSON list
def jsonl_to_json_list(jsonl_file_path):
    json_list = []
    with open(jsonl_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            json_obj = json.loads(line.strip())  # Parse each line as JSON
            json_list.append(json_obj)
    
    return json_list

# Save the JSON list to a file
def save_as_json(json_list, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(json_list, outfile, indent=4)

def save_as_jsonl(json_list, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        for json_obj in json_list:
            json.dump(json_obj, outfile)
            outfile.write('\n')

In [3]:
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def deduplicate_data(data):
    seen = set()
    deduplicated_data = []
    for item in data:
        idx = item['realidx']
        if idx not in seen:
            deduplicated_data.append(item)
            seen.add(idx)
    return deduplicated_data

def calculate_accuracy(data):
    correct_predictions = 0
    total_predictions = len(data)
    for item in data:
        if item['predicted_answer'] == item['answer_idx']:
            correct_predictions += 1
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

def calculate_cost_from_token_usage(data, model):
    total_cost = 0
    for item in data:
        if model == 'gpt-4o-mini':
            total_cost += item['token_usage']['prompt_tokens'] * 0.15 / 1000000 + item['token_usage']['completion_tokens'] * 0.6 / 1000000
        elif model == 'gpt-4o':
            total_cost += item['token_usage']['prompt_tokens'] * 2.5 / 1000000 + item['token_usage']['completion_tokens'] * 10 / 1000000
    return total_cost / len(data)

def calculate_time_from_data(data):
    total_time = 0
    for item in data:
        total_time += item['time_elapsed']
    return total_time / len(data)

tasks = {
    'medqa': ['test_hard'],
    'pubmedqa': ['test_hard'],
    'medmcqa': ['test_hard'],
    'medbullets': ['test_hard'],
    'mmlu': ['test_hard'],
    'mmlu-pro': ['test_hard'],
    # 'afrimedqa': ['test_hard'],
    'medexqa': ['test_hard'],
    'medxpertqa-u': ['test_hard'],
    'medxpertqa-r': ['test_hard'],
}
models = ['gpt-4o-mini', 'gpt-4o']
difficulties = ['adaptive']

print("| Model | Task | Subtask | Method | Accuracy | Cost per sample(USD) | Time per sample(s) | Total Number |")
print("|-------|------|---------|---------|----------|---------------------|-------------------|--------------|")
table = PrettyTable()
table.field_names = ["Model", "Task", "Subtask", "Difficulty", "Accuracy", "Cost per sample(USD)", "Time per sample(s)", "Total Number"]

total_cost = 0

for task in tasks:
    for model in models:
        for subtask in tasks[task]:
            for difficulty in difficulties:
                try:
                    file_path = f'./output/{task}/{model}_{task}_{subtask}_{difficulty}.json'
                    data = load_json(file_path)
                    output_path = f'../../output/{task}/{model if model != "deepseek-V3" else "DeepSeek-V3"}-{task}-{subtask}-mdagents.json'
                    save_as_json(data, output_path)
                    deduplicated_data = deduplicate_data(data)
                    accuracy = calculate_accuracy(deduplicated_data)
                    total = len(deduplicated_data)
                    cost_per_sample = calculate_cost_from_token_usage(deduplicated_data, model)
                    total_cost += cost_per_sample * total
                    print(f"| {model} | {task} | {subtask} | MDAgents | {accuracy * 100:.1f}% | {cost_per_sample} | {calculate_time_from_data(deduplicated_data)} | {total} |")
                    table.add_row([
                        model, task, subtask, difficulty,
                        f"{accuracy * 100:.1f}%",
                        cost_per_sample,
                        calculate_time_from_data(deduplicated_data),
                        total
                    ])
                except Exception as e:
                    print(f"Error loading file {file_path}: {e}")

print(table)
print(f"\nTotal cost of experiment: ${total_cost:.2f}")

| Model | Task | Subtask | Method | Accuracy | Cost per sample(USD) | Time per sample(s) | Total Number |
|-------|------|---------|---------|----------|---------------------|-------------------|--------------|
| gpt-4o-mini | medqa | test_hard | MDAgents | 24.0% | 0.012930928500000001 | 107.99290036201477 | 100 |
| gpt-4o | medqa | test_hard | MDAgents | 37.0% | 0.12832022500000004 | 86.27678339242935 | 100 |
| gpt-4o-mini | pubmedqa | test_hard | MDAgents | 22.0% | 0.009075352500000002 | 129.77324981451034 | 100 |
| gpt-4o | pubmedqa | test_hard | MDAgents | 21.0% | 0.12003512499999996 | 161.46251834630965 | 100 |
| gpt-4o-mini | medmcqa | test_hard | MDAgents | 23.2% | 0.007354648484848485 | 50.12014580977083 | 99 |
| gpt-4o | medmcqa | test_hard | MDAgents | 26.0% | 0.013982599999999998 | 14.395733473300934 | 100 |
| gpt-4o-mini | medbullets | test_hard | MDAgents | 14.6% | 0.007856462359550561 | 93.61956213565355 | 89 |
| gpt-4o | medbullets | test_hard | MDAgents | 13.5% | 0.0605